# 量化计算的基本认识

介绍：在大模型推理场景中，量化技术的应用及其优势，包括降低显存占用、减少计算量以及优化数据传输开销。文章将围绕以下两个基础内容展开：量化误差的产生机制与量化计算的基本过程。

相关文章：[大模型推理量化(Quantiztion)基础速览](https://zhuanlan.zhihu.com/p/2005335401469083798)

Author: kaiyuan

Email: kyxie@zju.edu.cn

## 1 量化误差的产生机制

演示并对比INT8和FP8 (E4M3)两种量化格式的量化误差。定义了量化和反量化函数，并使用一组测试数据来计算和展示不同量化方案下的原始值、量化值、反量化值以及量化误差。

INT8 量化误差分析：

- 对于小数值（如 0.001, 0.123, 1.234），INT8 量化能较好地保留精度，误差相对较小。
- 当原始值接近或超出INT8的表示范围（-128到127乘以缩放因子0.1，即-12.8到12.7）时，量化误差显著增大。例如，原始值127.9、255.5、-300.0、448.0、-448.0，由于被截断到INT8的最大或最小值（12.7或-12.8），导致反量化后的值与原始值偏差巨大，误差非常高。

FP8 (E4M3) 量化误差分析：
- 对于小数值，FP8 (E4M3)也表现出相似的小误差。
- FP8 (E4M3)的表示范围相对INT8更大（最大448乘以缩放因子0.1，即44.8）。因此，对于像127.9这样的值，它虽然也被截断，但截断发生在更大的范围，其反量化误差（83.1）比INT8（115.2）要小。
- 对于超出FP8 (E4M3)表示范围的值（例如255.5，反量化上限是44.8），同样会发生溢出截断，导致较大的误差。

两种量化方法在处理其有效表示范围内的值时，都能提供一定的精度。但当原始值超出其各自的表示范围时，都会发生严重的量化误差，尤其是在截断到最大或最小值时。FP8(E4M3)由于其指数部分的特性，通常能表示更大的动态范围，因此在某些情况下可以比定点INT8减少溢出带来的误差，但其精度粒度可能不如INT8均匀。

In [ ]:
# -*- coding: gbk -*-
import numpy as np

# 定义量化和反量化函数
def quantize_int8(value, scale_factor):
    """
    INT8量化：将浮点数映射到INT8范围（-128到127）。
    """
    quantized_value = int(np.round(value / scale_factor))
    quantized_value = np.clip(quantized_value, -128, 127)  # 限制在INT8范围内
    return quantized_value

def dequantize_int8(quantized_value, scale_factor):
    """
    INT8反量化：将量化后的整数还原为浮点数。
    """
    return quantized_value * scale_factor

def quantize_fp8_e4m3(value, scale_factor):
    """
    FP8 (E4M3)量化：将浮点数映射到FP8 (E4M3)格式。
    """
    # 将值转换为无符号浮点数
    unscaled_value = value / scale_factor
    # 模拟FP8 (E4M3)的表示范围
    if abs(unscaled_value) > 448:
        return np.sign(unscaled_value) * 448  # 溢出处理
    return np.round(unscaled_value)

def dequantize_fp8_e4m3(quantized_value, scale_factor):
    """
    FP8 (E4M3)反量化：将量化后的值还原为浮点数。
    """
    return quantized_value * scale_factor

# 测试数据
values = [
    0.001,  # 非常接近零的小数
    0.123,  # 小数
    1.234,  # 小数
    127.9,  # 接近 INT8 上限
    255.5,  # 超出 INT8 范围
    -300.0,  # 负数且超出 INT8 范围
    448.0,  # FP8 的最大值
    -448.0  # FP8 的最小值
]

# 缩放因子
scale_factor = 0.1

# 打印标题
print("原始值\tINT8量化值\tINT8反量化值\tINT8误差")
for value in values:
    # INT8量化与反量化
    int8_quantized = quantize_int8(value, scale_factor)
    int8_dequantized = dequantize_int8(int8_quantized, scale_factor)
    int8_error = abs(value - int8_dequantized)
    print(f"{value:.5f}\t{int8_quantized}\t{int8_dequantized:.5f}\t{int8_error:.5f}")

print("\n原始值\tFP8 (E4M3)量化值\tFP8 (E4M3)反量化值\tFP8 (E4M3)误差")
for value in values:
    # FP8 (E4M3)量化与反量化
    fp8_quantized = quantize_fp8_e4m3(value, scale_factor)
    fp8_dequantized = dequantize_fp8_e4m3(fp8_quantized, scale_factor)
    fp8_error = abs(value - fp8_dequantized)
    print(f"{value:.5f}\t{fp8_quantized}\t{fp8_dequantized:.5f}\t{fp8_error:.5f}")

原始值	INT8量化值	INT8反量化值	INT8误差
0.00100	0	0.00000	0.00100
0.12300	1	0.10000	0.02300
1.23400	12	1.20000	0.03400
127.90000	127	12.70000	115.20000
255.50000	127	12.70000	242.80000
-300.00000	-128	-12.80000	287.20000
448.00000	127	12.70000	435.30000
-448.00000	-128	-12.80000	435.20000

原始值	FP8 (E4M3)量化值	FP8 (E4M3)反量化值	FP8 (E4M3)误差
0.00100	0.0	0.00000	0.00100
0.12300	1.0	0.10000	0.02300
1.23400	12.0	1.20000	0.03400
127.90000	448.0	44.80000	83.10000
255.50000	448.0	44.80000	210.70000
-300.00000	-448.0	-44.80000	255.20000
448.00000	448.0	44.80000	403.20000
-448.00000	-448.0	-44.80000	403.20000


## 2 量化计算过程演示

矩阵计算： c = a * b 其中a，b为int类型，b为fp31。

采用Trition库完成运算，本例需要使用GPU，建议采用docker容器。
```
docker pull nvcr.io/nvidia/sglang:26.01-py3
```

测试机器信息：
- NVIDIA A100-SXM4-80GB
- NVIDIA-SMI 570.172.08
- Driver Version: 570.172.08
- CUDA Version: 13.1


说明：

1. **数据类型流动**：
    *   **输入**：`torch.int8`。
    *   **计算**：`tl.dot(a, b)` 内部是 `INT8 * INT8 -> INT32`。我们立即 `.to(tl.float32)` 并累加到FP32累加器 `acc` 中。
    *   **输出**：FP32，乘以缩放因子后直接存储。

2. **为什么中间用INT32，不直接在FP32里乘？**
    *   虽然可以直接把INT8 load成FP32再相乘，但会浪费Tensor Core的INT8算力。标准做法是用`tl.dot`输出INT32再转FP32，兼顾精度与速度。

3. **量化缩放**：
    *   这是INT8推理的标准流程：A_int8 * W_int8 = Y_int32，然后 `Y_fp32 = Y_int32 * scale_a * scale_b`。
    *   代码中`scale_a_ptr`和`scale_b_ptr`是标量指针，演示per-tensor缩放。扩展到per-token/per-channel只需改为向量加载。

In [ ]:
# -*- coding: gbk -*-
import torch
import triton
import triton.language as tl
import numpy as np

# ------------------------------------------------------------
# 高性能分块版本：INT8 输入，FP32 输出（带量化缩放）
# 每个线程块负责计算一个 [BLOCK_M, BLOCK_N] 的输出分块
# ------------------------------------------------------------
@triton.jit
def int8_gemm_tiled_kernel(
    # 指针
    a_ptr, b_ptr, c_ptr,
    scale_a_ptr, scale_b_ptr,  # 每张量缩放因子（FP32）
    M, N, K,
    stride_am, stride_ak,
    stride_bk, stride_bn,
    stride_cm, stride_cn,
    # 元参数：分块大小
    BLOCK_M: tl.constexpr, BLOCK_N: tl.constexpr, BLOCK_K: tl.constexpr,
):
    # 线程块在输出矩阵中的位置
    pid_m = tl.program_id(0)
    pid_n = tl.program_id(1)

    # 该块负责的行的范围
    offs_m = pid_m * BLOCK_M + tl.arange(0, BLOCK_M)
    offs_n = pid_n * BLOCK_N + tl.arange(0, BLOCK_N)
    offs_k = tl.arange(0, BLOCK_K)

    # 累加器：FP32！直接以FP32累加，避免后续类型转换开销
    acc = tl.zeros((BLOCK_M, BLOCK_N), dtype=tl.float32)

    # 遍历 K 维度，每次处理 BLOCK_K 个元素
    for k in range(0, K, BLOCK_K):
        # ---- 1. 创建 A 的分块指针 (INT8) ----
        a_ptrs = a_ptr + (offs_m[:, None] * stride_am + (k + offs_k[None, :]) * stride_ak)
        # ---- 2. 创建 B 的分块指针 (INT8) ----
        b_ptrs = b_ptr + ((k + offs_k[:, None]) * stride_bk + offs_n[None, :] * stride_bn)

        # 加载 INT8 数据，自动提升为 INT32 供 tl.dot 使用
        a = tl.load(a_ptrs, mask=(offs_m[:, None] < M) & ((k + offs_k[None, :]) < K), other=0)
        b = tl.load(b_ptrs, mask=((k + offs_k[:, None]) < K) & (offs_n[None, :] < N), other=0)

        # ---- 3. 核心矩阵乘：INT8 * INT8 -> INT32 ----
        # tl.dot 要求输入至少是 INT16，这里 INT8 会自动扩展，输出 INT32
        acc += tl.dot(a, b).to(tl.float32)  # 关键：INT32 转 FP32 后累加

    # ---- 4. 量化反量化：应用缩放因子，得到最终 FP32 输出 ----
    scale_a = tl.load(scale_a_ptr)  # per-tensor 激活缩放
    scale_b = tl.load(scale_b_ptr)  # per-tensor 权重缩放
    c = acc * (scale_a * scale_b)   # 公式：C_fp32 = (A_int8 * B_int8) * (scale_a * scale_b)

    # ---- 5. 存储 FP32 结果 ----
    c_ptrs = c_ptr + offs_m[:, None] * stride_cm + offs_n[None, :] * stride_cn
    tl.store(c_ptrs, c, mask=(offs_m[:, None] < M) & (offs_n[None, :] < N))


def int8_gemm_fp32_output(a_int8, b_int8, scale_a=1.0, scale_b=1.0):
    """
    INT8 矩阵乘，FP32 输出
    - a_int8, b_int8: shape (M, K), (K, N)，torch.int8，CUDA
    - scale_a, scale_b: 每张量缩放因子，FP32，用于反量化
    - 返回: torch.float32 矩阵，值为 (a_int8 * b_int8) * (scale_a * scale_b)
    """
    assert a_int8.is_cuda and b_int8.is_cuda
    assert a_int8.dtype == torch.int8 and b_int8.dtype == torch.int8
    M, K = a_int8.shape
    K_, N = b_int8.shape
    assert K == K_

    # 输出 FP32
    c_fp32 = torch.empty((M, N), device=a_int8.device, dtype=torch.float32)

    # 缩放因子作为标量张量传入内核
    scale_a_t = torch.tensor(scale_a, device=a_int8.device, dtype=torch.float32)
    scale_b_t = torch.tensor(scale_b, device=a_int8.device, dtype=torch.float32)

    # 分块大小（可调）
    BLOCK_M, BLOCK_N, BLOCK_K = 64, 64, 32
    grid = (triton.cdiv(M, BLOCK_M), triton.cdiv(N, BLOCK_N))

    int8_gemm_tiled_kernel[grid](
        a_int8, b_int8, c_fp32,
        scale_a_t, scale_b_t,
        M, N, K,
        a_int8.stride(0), a_int8.stride(1),
        b_int8.stride(0), b_int8.stride(1),
        c_fp32.stride(0), c_fp32.stride(1),
        BLOCK_M, BLOCK_N, BLOCK_K,
    )
    return c_fp32


# ------------------------------------------------------------
# 测试验证
# ------------------------------------------------------------
def test_int8_gemm_fp32():
    torch.manual_seed(42)
    M, N, K = 128, 128, 64   # 稍微放大，让分块效果更明显

    # 随机 INT8 矩阵（范围 -128~127）
    a = torch.randint(-128, 127, (M, K), device='cuda', dtype=torch.int8)
    b = torch.randint(-128, 127, (K, N), device='cuda', dtype=torch.int8)

    # 随机缩放因子（模拟量化反量化）
    scale_a = np.random.uniform(0.01, 0.1)
    scale_b = np.random.uniform(0.01, 0.1)

    # Triton FP32 输出
    c_triton = int8_gemm_fp32_output(a, b, scale_a, scale_b)

    # NumPy 参考：INT8 -> FP32 -> 乘缩放
    a_np = a.cpu().numpy().astype(np.float32)
    b_np = b.cpu().numpy().astype(np.float32)
    c_np = (a_np @ b_np) * (scale_a * scale_b)

    # 误差容忍度（INT8 量化本身有舍入误差）
    torch.testing.assert_close(c_triton.cpu(), torch.from_numpy(c_np), rtol=1e-2, atol=1e-2)
    print("测试通过：INT8 输入，FP32 输出，带量化缩放")

if __name__ == "__main__":
    test_int8_gemm_fp32()

测试通过：INT8 输入，FP32 输出，带量化缩放
